In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
# LOAD DATA
train_pro = pd.read_csv("train_proteins.csv")
train_pep= pd.read_csv("train_peptides.csv")
train_clin = pd.read_csv("train_clinical_data.csv")

In [6]:
# VISUALIZE DATA TO JOIN DATA
#-> .head() , .describe(), .info(), .shape()

In [7]:
print(train_pro.shape)
print(train_pep.shape)
print(train_clin.shape)

(232741, 5)
(981834, 6)
(2615, 8)


In [21]:
# different amount of columns, thus need to find similarities and differences bfr making a MERGE
print(train_pro.head(3000))
print(train_pep.head())
print(train_clin.head())

     visit_id  visit_month  patient_id UniProt         NPX
0        55_0            0          55  O00391    11254.30
1        55_0            0          55  O00533   732430.00
2        55_0            0          55  O00584    39585.80
3        55_0            0          55  O14498    41526.90
4        55_0            0          55  O14773    31238.00
...       ...          ...         ...     ...         ...
2995   7051_0            0        7051  P01877   264629.00
2996   7051_0            0        7051  P02452     6669.77
2997   7051_0            0        7051  P02647  8695510.00
2998   7051_0            0        7051  P02649  3950710.00
2999   7051_0            0        7051  P02652   628789.00

[3000 rows x 5 columns]
  visit_id  visit_month  patient_id UniProt  \
0     55_0            0          55  O00391   
1     55_0            0          55  O00533   
2     55_0            0          55  O00533   
3     55_0            0          55  O00533   
4     55_0            0         

In [9]:
#All have: Visit ID, visit MONTH, Patient ID, 
#BUT NOT : UNI PROT, NPX, PEPTIDE, PEPTIDE ABUNDANCE, UPDRS1-4, ON MEDICATION
#WANT: Visit ID, NPX, PEPTIDE ABUNDANCE,UPDRS1-4

In [27]:
#Clean Data To Merge Data
# Drop Month ID since 0 value and does not contribute to data processing

pro_Cond1= train_pro.drop("visit_month", axis=1 )
pep_Cond1= train_pep.drop("visit_month", axis=1 )

pro_Cond1.head()


,visit_id,patient_id,UniProt,NPX
0,55_0,55,O00391,11254.3
1,55_0,55,O00533,732430.0
2,55_0,55,O00584,39585.8
3,55_0,55,O14498,41526.9
4,55_0,55,O14773,31238.0


In [28]:
pep_Cond1.head()

,visit_id,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,55,O00533,SMEQNGPGLEYR,30838.7


In [36]:
#MERGE(JOIN)DATA Part 1
Pro_pep_cond=pro_Cond1.merge(pep_Cond1, on =["visit_id", "UniProt"])
Pro_pep_cond.head()

,visit_id,patient_id_x,UniProt,NPX,patient_id_y,Peptide,PeptideAbundance
0,55_0,55,O00391,11254.3,55,NEQEQPLGQWHLS,11254.3
1,55_0,55,O00533,732430.0,55,GNPEPTFSWTK,102060.0
2,55_0,55,O00533,732430.0,55,IEIPSSVQQVPTIIK,174185.0
3,55_0,55,O00533,732430.0,55,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,55,O00533,732430.0,55,SMEQNGPGLEYR,30838.7


In [40]:
#MERGE(JOIN)DATA Part 2
Pro_pep_clin= Pro_pep_cond.merge(train_clin, on =["visit_id"])
Pro_pep_clin.head()

,visit_id,patient_id_x,UniProt,NPX,patient_id_y,Peptide,PeptideAbundance,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,O00391,11254.3,55,NEQEQPLGQWHLS,11254.3,55,0,10.0,6.0,15.0,NaN,NaN
1,55_0,55,O00533,732430.0,55,GNPEPTFSWTK,102060.0,55,0,10.0,6.0,15.0,NaN,NaN
2,55_0,55,O00533,732430.0,55,IEIPSSVQQVPTIIK,174185.0,55,0,10.0,6.0,15.0,NaN,NaN
3,55_0,55,O00533,732430.0,55,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9,55,0,10.0,6.0,15.0,NaN,NaN
4,55_0,55,O00533,732430.0,55,SMEQNGPGLEYR,30838.7,55,0,10.0,6.0,15.0,NaN,NaN
